In [16]:
# Importar las librerías necesarias
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Cargar el conjunto de datos Iris
iris = load_iris()
X = iris.data  # Características
y = iris.target  # Etiquetas

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, 
)

# Crear una instancia del clasificador KNN
k = 3
knn = KNeighborsClassifier(n_neighbors=k)

# Entrenar el modelo
knn.fit(X_train, y_train)

# Realizar predicciones
y_pred = knn.predict(X_test)

# Calcular la precisión del modelo
accuracy = knn.score(X_test, y_test)

# Imprimir la precisión del modelo
print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 0.97


In [17]:
!pip install skl2onnx onnxmltools onnxruntime --quiet

In [18]:
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Definir el tipo de entrada del modelo
initial_type = [("float_input", FloatTensorType([None, X_train.shape[1]]))]

# Convertir el modelo a formato ONNX
onnx_model = convert_sklearn(knn, initial_types=initial_type)

# Guardar el modelo ONNX en un archivo
with open("knn_iris.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [19]:
import onnxruntime as rt
import numpy as np

# Cargar el modelo ONNX
sess = rt.InferenceSession("knn_iris.onnx")

# Obtener el nombre de la entrada y salida del modelo
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

# Realizar predicciones en el conjunto de prueba
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

# Evaluar el rendimiento del modelo
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, pred_onx)
print(f"Precisión del modelo ONNX KNN: {accuracy:.2f}")

Precisión del modelo ONNX KNN: 0.97
